## Desafio 1


Dada uma base de dados de imagens de pessoas da [Caltech](http://www.vision.caltech.edu/html-files/archive.html) na pasta "caltech", colete os dados de cada pessoa, das 10 disponíveis, identificado por meio do seu id ("Subject 1", "Subject 2"  e assim por diante). Use a pasta "train" para treinar um modelo de classificador de rostos e valide os resultados com outras amostras que estão na pasta "val".

Após os testes, teste seu modelo com 10 imagens na pasta "imagens" e avalie como o classificador escolhido se comportou.

Utilize como referência dos sujeitos a ordem a seguir.

Sujeito 1
<img src="imagens/caltech/train/1/image_0002.jpg" width="40%" height="40%">

Sujeito 2
<img src="imagens/caltech/train/2/image_0023.jpg" width="40%" height="40%">

Sujeito 3
<img src="imagens/caltech/train/3/image_0048.jpg" width="40%" height="40%">

Sujeito 4
<img src="imagens/caltech/train/4/image_0070.jpg" width="40%" height="40%">

Sujeito 5
<img src="imagens/caltech/train/5/image_0091.jpg" width="40%" height="40%">

Sujeito 6
<img src="imagens/caltech/train/6/image_0114.jpg" width="40%" height="40%">

Sujeito 7
<img src="imagens/caltech/train/7/image_0139.jpg" width="40%" height="40%">

Sujeito 8
<img src="imagens/caltech/train/8/image_0197.jpg" width="40%" height="40%">

Sujeito 9
<img src="imagens/caltech/train/9/image_0243.jpg" width="40%" height="40%">

Sujeito 10
<img src="imagens/caltech/train/10/image_0429.jpg" width="40%" height="40%">

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob

In [2]:
face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

# Extrator de faces
def face_extractor(img):
    try:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if faces is ():
            return None

        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            
    except:
        return None

    return cropped_face

In [32]:
def normalizar_imagens(subject):
    list_files = []
    i = 1
    no_face = 0
    total = 0
    for file in glob.glob("imagens/caltech/train/"+str(subject)+"/*"):
        total+=1
        image = cv2.imread(file);
        face = face_extractor(image)
        if face is not None:
            image_norm = cv2.resize(face, (200, 200))
            image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
            file_name_path = "imagens/caltech/train/"+str(subject)+"/norm/"+str(i)+".png"
            cv2.imwrite(file_name_path, image_norm)
            list_files.append(file_name_path)
            i += 1
        else:
            no_face += 1
    print("Faces localizadas "+str(i) + "/" + str(total))
    return list_files

In [39]:
def treinamento():
    training_data, labels = [], []

    for subject in list(range(1,11)):
        print("Processando "+str(subject))
        files = normalizar_imagens(subject)

        # Lendo as imagens e associando a um label
        for i, file in enumerate(files):
            images = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            training_data.append(np.asarray(images, dtype=np.uint8))
            labels.append(subject)

    # Criando uma matriz da lista de labels
    labels = np.asarray(labels, dtype=np.int32)
    return training_data, labels

In [40]:
training_data, labels = treinamento()

Processando 1
Faces localizadas 16/16
Processando 2
Faces localizadas 15/15
Processando 3
Faces localizadas 17/17
Processando 4
Faces localizadas 16/16
Processando 5
Faces localizadas 17/17
Processando 6
Faces localizadas 15/15
Processando 7
Faces localizadas 16/16
Processando 8
Faces localizadas 16/16
Processando 9
Faces localizadas 17/17
Processando 10
Faces localizadas 17/17


In [41]:
labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [42]:
# Treinamento do modelo
model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(training_data), labels)
print("Modelo treinado com sucesso.")

Modelo treinado com sucesso.


In [48]:
i = 1
no_face = 0
for subject in list(range(1,11)):
    print("Analisando "+ str(subject))
    total = 0
    success = 0
    for file in glob.glob("imagens/caltech/val/"+str(subject)+"/*"):
        total += 1
        image = cv2.imread(file);
        face = face_extractor(image)
        if face is not None:
            image_norm = cv2.resize(face, (200, 200))
            image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
            results = model.predict(image_norm)
            info = str(results[0]) + " " + str(int(results[1]))+" pontos"
            print(info)
            if results[0] == subject:
                success += 1
        else:
            no_face += 1
    print("Faces localizadas "+str(success) + "/" + str(total)+ " "+str(float(success)*100/float(total)) + "%")

Analisando 1
1 34 pontos
1 36 pontos
1 37 pontos
1 38 pontos
1 47 pontos
Faces localizadas 5/5 100.0%
Analisando 2
2 38 pontos
2 36 pontos
2 39 pontos
2 40 pontos
2 39 pontos
Faces localizadas 5/5 100.0%
Analisando 3
3 35 pontos
3 37 pontos
3 37 pontos
3 36 pontos
3 37 pontos
Faces localizadas 5/5 100.0%
Analisando 4
4 34 pontos
4 33 pontos
4 34 pontos
4 33 pontos
4 36 pontos
Faces localizadas 5/5 100.0%
Analisando 5
5 36 pontos
5 35 pontos
5 35 pontos
5 37 pontos
2 70 pontos
Faces localizadas 4/5 80.0%
Analisando 6
6 32 pontos
6 29 pontos
6 34 pontos
6 36 pontos
6 31 pontos
Faces localizadas 5/5 100.0%
Analisando 7
7 36 pontos
7 40 pontos
7 36 pontos
7 37 pontos
7 36 pontos
Faces localizadas 5/5 100.0%
Analisando 8
8 41 pontos
8 36 pontos
8 36 pontos
8 38 pontos
8 36 pontos
Faces localizadas 5/5 100.0%
Analisando 9
9 37 pontos
9 36 pontos
9 33 pontos
9 38 pontos
9 36 pontos
Faces localizadas 5/5 100.0%
Analisando 10
10 35 pontos
10 33 pontos
10 34 pontos
10 36 pontos
10 35 pontos
Face

In [59]:
for file in sorted(glob.glob("imagens/*.jpg")):
    print("Processando arquivo " + file + "...")
    image = cv2.imread(file);
    face = face_extractor(image)
    if face is not None:
        image_norm = cv2.resize(face, (200, 200))
        image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
        results = model.predict(image_norm)
        info = "Subject " + str(results[0]) + " " + str(int(results[1]))+" pontos"
        cv2.putText(image, info, (10,40), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
        cv2.imshow("Imagem Final com Digitos", image)
        cv2.waitKey(0) 
        cv2.destroyAllWindows()
        print(info)
    else:
        print("Não encontrado faces")

Processando arquivo imagens\1.jpg...
Subject 1 38 pontos
Processando arquivo imagens\10.jpg...
Subject 10 33 pontos
Processando arquivo imagens\2.jpg...
Subject 2 36 pontos
Processando arquivo imagens\3.jpg...
Subject 3 36 pontos
Processando arquivo imagens\4.jpg...
Subject 4 34 pontos
Processando arquivo imagens\5.jpg...
Subject 5 36 pontos
Processando arquivo imagens\6.jpg...
Subject 6 30 pontos
Processando arquivo imagens\7.jpg...
Subject 7 35 pontos
Processando arquivo imagens\8.jpg...
Subject 8 40 pontos
Processando arquivo imagens\9.jpg...
Subject 9 35 pontos
